In [8]:
# Import Libraries
import numpy as np
import pandas as pd
import requests
import random
import time
from io import StringIO
from sqlalchemy import create_engine

In [9]:
#Create list of team codes 
teams = [
    'ATL', 'BOS', 'BRK', 'CHO', 'CHI', 'CLE', 'DAL', 'DEN', 'DET', 'GSW', 
    'HOU', 'IND', 'LAC', 'LAL', 'MEM', 'MIA', 'MIL', 'MIN', 'NOP', 'NYK', 
    'OKC', 'ORL', 'PHI', 'PHO', 'POR', 'SAC', 'SAS', 'TOR', 'UTA', 'WAS'
]

In [10]:
#Create code: team name dictionary
team_dict = {
    'ATL': 'Hawks', 'BOS': 'Celtics', 'BRK': 'Nets', 'CHO': 'Hornets', 'CHI': 'Bulls', 'CLE': 'Cavaliers',
    'DAL': 'Mavericks', 'DEN': 'Nuggets', 'DET': 'Pistons', 'GSW': 'Warriors', 'HOU': 'Rockets', 'IND': 'Pacers', 
    'LAC': 'Clippers', 'LAL': 'Lakers', 'MEM': 'Grizzlies', 'MIA': 'Heat', 'MIL': 'Bucks', 'MIN': 'Timberwolves', 
    'NOP': 'Pelicans', 'NYK': 'Knicks', 'OKC': 'Thunder', 'ORL': 'Magic', 'PHI': '76ers', 'PHO': 'Suns', 
    'POR': 'Trail Blazers', 'SAC': 'Kings', 'SAS': 'Spurs', 'TOR': 'Raptors', 'UTA': 'Jazz', 'WAS': 'Wizards'
}

In [11]:
#Create the list of years (seasons)
seasons = ['2022', '2023']


In [12]:
# Create the list of years (seasons)
seasons = ['2022', '2023']

# Create an empty DataFrame
nba_df = pd.DataFrame()

# Function to fetch and process data for a team in a season
def fetch_team_data(team, season):
    # Set URL
    url = f'https://www.basketball-reference.com/teams/{team}/{season}/gamelog-advanced/'
    print(f"Fetching data for {team_dict[team]} in {season}: {url}")
    
    # Get a response object
    response = requests.get(url, headers={'User-agent': 'Mozilla/5.0'})
    visible_html = response.text.replace('<!--', '')
    
    # Get DataFrames
    df_list = pd.read_html(StringIO(visible_html), header=1)
    
    # Use only the regular-season DataFrame
    if len(df_list) >= 1:
        team_df = df_list[0]
        
        # Drop rows where 'G' is NaN or 'G'
        team_df = team_df.dropna(subset=['G'])
        team_df = team_df[team_df['G'] != 'G']
        team_df = team_df.reset_index(drop=True)
        
        # Keep relevant columns
        team_df = team_df.iloc[:, 1:11]
        
        # Rename columns
        team_df = team_df.rename(columns={'Unnamed: 3': 'At', 'Tm': 'OPts', 'Opp.1': 'DPts'})
        
        # Process 'At' column
        team_df['At'] = team_df['At'].apply(lambda x: x if x == '@' else "")
        team_df['Location'] = team_df['At'].apply(lambda x: 'Away' if x == '@' else 'Home')
        team_df = team_df.drop(columns=['At'])
        
        # Process 'W/L' column
        team_df['Win'] = team_df['W/L'].apply(lambda x: 1 if x == 'W' else 0)
        team_df = team_df.drop(columns=['W/L'])
        
        # Add additional columns
        team_df.insert(0, 'Season', season)
        team_df.insert(2, 'Team', team)
        team_df.insert(3, 'Name', team_dict[team])
        team_df['Opp_Name'] = team_df['Opp'].apply(lambda x: team_dict.get(x, x))
        team_df.insert(8, 'Opp_Name', team_df.pop('Opp_Name'))
        
        # Convert data types
        numeric_columns = ['G', 'OPts', 'DPts', 'ORtg', 'DRtg', 'Pace']
        for col in numeric_columns:
            team_df[col] = pd.to_numeric(team_df[col], errors='coerce')
        
        return team_df
    else:
        print(f"No regular-season data for {team} in {season}")
        return pd.DataFrame()  # Return empty DataFrame if no data

# Iterate through seasons and teams
for season in seasons:
    for team in teams:
        try:
            team_df = fetch_team_data(team, season)
            if not team_df.empty:
                # Aggregate the DataFrame
                nba_df = pd.concat([nba_df, team_df], ignore_index=True)
            # Pause to abide by basketball-reference.com rules
            time.sleep(random.randint(4, 6))
        except Exception as e:
            print(f"Error processing {team} in {season}: {e}")
            continue

# Display the DataFrame
print(nba_df.shape)
print(nba_df.head())

Fetching data for Hawks in 2022: https://www.basketball-reference.com/teams/ATL/2022/gamelog-advanced/
Fetching data for Celtics in 2022: https://www.basketball-reference.com/teams/BOS/2022/gamelog-advanced/
Fetching data for Nets in 2022: https://www.basketball-reference.com/teams/BRK/2022/gamelog-advanced/
Fetching data for Hornets in 2022: https://www.basketball-reference.com/teams/CHO/2022/gamelog-advanced/
Fetching data for Bulls in 2022: https://www.basketball-reference.com/teams/CHI/2022/gamelog-advanced/
Fetching data for Cavaliers in 2022: https://www.basketball-reference.com/teams/CLE/2022/gamelog-advanced/
Fetching data for Mavericks in 2022: https://www.basketball-reference.com/teams/DAL/2022/gamelog-advanced/
Fetching data for Nuggets in 2022: https://www.basketball-reference.com/teams/DEN/2022/gamelog-advanced/
Fetching data for Pistons in 2022: https://www.basketball-reference.com/teams/DET/2022/gamelog-advanced/
Fetching data for Warriors in 2022: https://www.basketball

In [13]:
print(nba_df)

     Season   G Team     Name        Date  Opp  OPts  DPts   Opp_Name   ORtg  \
0      2022   1  ATL    Hawks  2021-10-21  DAL   113    87  Mavericks  109.9   
1      2022   2  ATL    Hawks  2021-10-23  CLE    95   101  Cavaliers   99.3   
2      2022   3  ATL    Hawks  2021-10-25  DET   122   104    Pistons  123.1   
3      2022   4  ATL    Hawks  2021-10-27  NOP   102    99   Pelicans  110.2   
4      2022   5  ATL    Hawks  2021-10-28  WAS   111   122    Wizards  111.9   
...     ...  ..  ...      ...         ...  ...   ...   ...        ...    ...   
4915   2023  78  WAS  Wizards  2023-04-02  NYK   109   118     Knicks  110.8   
4916   2023  79  WAS  Wizards  2023-04-04  MIL   128   140      Bucks  117.9   
4917   2023  80  WAS  Wizards  2023-04-05  ATL   116   134      Hawks  111.6   
4918   2023  81  WAS  Wizards  2023-04-07  MIA   114   108       Heat  111.3   
4919   2023  82  WAS  Wizards  2023-04-09  HOU   109   114    Rockets  106.5   

       DRtg   Pace Location  Win  
0   

In [14]:
# Update data types (if not done in the function)
nba_df['G'] = nba_df['G'].astype(int)
nba_df['OPts'] = nba_df['OPts'].astype(int)
nba_df['DPts'] = nba_df['DPts'].astype(int)
nba_df['ORtg'] = nba_df['ORtg'].astype(float)
nba_df['DRtg'] = nba_df['DRtg'].astype(float)
nba_df['Pace'] = nba_df['Pace'].astype(float)

In [15]:
print(nba_df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4920 entries, 0 to 4919
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Season    4920 non-null   object 
 1   G         4920 non-null   int32  
 2   Team      4920 non-null   object 
 3   Name      4920 non-null   object 
 4   Date      4920 non-null   object 
 5   Opp       4920 non-null   object 
 6   OPts      4920 non-null   int32  
 7   DPts      4920 non-null   int32  
 8   Opp_Name  4920 non-null   object 
 9   ORtg      4920 non-null   float64
 10  DRtg      4920 non-null   float64
 11  Pace      4920 non-null   float64
 12  Location  4920 non-null   object 
 13  Win       4920 non-null   int64  
dtypes: float64(3), int32(3), int64(1), object(7)
memory usage: 480.6+ KB
None


In [16]:
#SQLIte database
engine = create_engine('sqlite:///nba_advanced_gamelogs_22_23.db')
nba_df.to_sql('nba_stats', con=engine, if_exists='replace', index=False)

4920